### Kaggle TensorFlow Speech Competition
Use TensorFlow Simple Audio Recognition tutorial code to create submitssion file.

In [1]:
# download dataset
import os, sys, shutil
import pickle

NOTEBOOK_DIR = '/notebooks'
DATA_DIR = '/notebooks/data'
#TRAIN_DIR = DATA_DIR + '/training_atom_xavier_batchnorm'
TRAIN_DIR = DATA_DIR + '/training_pseudo_label'
#TRAIN_DIR = DATA_DIR + '/training_low_latency'
TEST_DIR = DATA_DIR + "/test/audio"
KAGGLE_TRAIN_DATA_DIR = DATA_DIR + '/train/audio'
PSEUDO_LABEL_FILE = DATA_DIR + '/test/pseudo_label.pickle'

if os.path.isdir(DATA_DIR):
    os.chdir(DATA_DIR)
else:
    os.chdir(NOTEBOOK_DIR)
    os.mkdir('data')
    os.chdir('data')

print (os.getcwd())

# Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# assume that the data is downloaded and extracted

# copy batchnorm trained model to pseudo_label directoriesshutil.rmtree(
'''
if os.path.exists(TRAIN_DIR):
    shutil.rmtree(TRAIN_DIR)
shutil.copytree(DATA_DIR + '/training_atom_xavier_batchnorm/speech_commands_train/',
                TRAIN_DIR + '/speech_commands_train')
'''

/notebooks/data


"\nif os.path.exists(TRAIN_DIR):\n    shutil.rmtree(TRAIN_DIR)\nshutil.copytree(DATA_DIR + '/training_atom_xavier_batchnorm/speech_commands_train/',\n                TRAIN_DIR + '/speech_commands_train')\n"

### Try speech_command sample

In [2]:
import argparse

import numpy as np
import pandas as pd
from six.moves import xrange
import tensorflow as tf

import input_data; reload (input_data)
import models
from tensorflow.python.platform import gfile

# Set up argument list
parser = argparse.ArgumentParser()
parser.add_argument('--data_url', type=str,
                    # pylint: disable=line-too-long
                    #default='http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                    default='',
                    # pylint: enable=line-too-long
                    help='Location of speech training data archive on the web.')
parser.add_argument('--data_dir', type=str, default=KAGGLE_TRAIN_DATA_DIR,
                    help="""\
                    Where to download the speech training data to.
                    """)
parser.add_argument('--background_volume', type=float, default=0.1,
                    help="""\
                    How loud the background noise should be, between 0 and 1.
                    """)
parser.add_argument('--background_frequency', type=float, default=0.8,
                    help="""\
                    How many of the training samples have background noise mixed in.
                    """)
parser.add_argument('--silence_percentage', type=float, default=10.0,
                    help="""\
                    How much of the training data should be silence.
                    """)
parser.add_argument('--unknown_percentage', type=float, default=10.0,
                    help="""\
                    How much of the training data should be unknown words.
                    """)
parser.add_argument('--time_shift_ms', type=float, default=100.0,
                    help="""\
                    Range to randomly shift the training audio by in time.
                    """)
parser.add_argument('--testing_percentage', type=int, default=10,
                    help='What percentage of wavs to use as a test set.')
parser.add_argument('--validation_percentage', type=int, default=10,
                    help='What percentage of wavs to use as a validation set.')
parser.add_argument('--sample_rate', type=int, default=16000,
                    help='Expected sample rate of the wavs',)
parser.add_argument('--clip_duration_ms', type=int, default=1000,
                    help='Expected duration in milliseconds of the wavs',)
parser.add_argument('--window_size_ms', type=float, default=30.0,
                    help='How long each spectrogram timeslice is',)
parser.add_argument('--window_stride_ms', type=float, default=10.0,
                    help='How long each spectrogram timeslice is',)
parser.add_argument('--dct_coefficient_count', type=int, default=40,
                    help='How many bins to use for the MFCC fingerprint',)
# default='15000,3000',
# default='1000,1000,1000,1000'
parser.add_argument('--how_many_training_steps', type=str, default='1500,1000,1000,1000',
                    help='How many training loops to run',)
parser.add_argument('--eval_step_interval', type=int, default=400,
                    help='How often to evaluate the training results.')
# default='0.001,0.0001,0.00001,0.000001'
parser.add_argument('--learning_rate', type=str, default='0.001,0.0001,0.00001,0.000001',
                    help='How large a learning rate to use when training.')
# default=100
parser.add_argument('--batch_size', type=int, default=500,
                    help='How many items to train with at once',)
parser.add_argument('--summaries_dir', type=str, default=TRAIN_DIR + '/retrain_logs',
                    help='Where to save summary logs for TensorBoard.')
parser.add_argument('--wanted_words', type=str, default='yes,no,up,down,left,right,on,off,stop,go',
                    help='Words to use (others will be added to an unknown label)',)
parser.add_argument('--train_dir', type=str, default=TRAIN_DIR + '/speech_commands_train',
                    help='Directory to write event logs and checkpoint.')
parser.add_argument('--save_step_interval', type=int, default=100,
                    help='Save model checkpoint every save_steps.')
#--start_checkpoint=/tmp/speech_commands_train/conv.ckpt-100 or TRAIN_DIR + '/speech_commands_train/conv.ckpt-4000'
parser.add_argument('--start_checkpoint', type=str, default='',
                    help='If specified, restore this pretrained model before any training.')
#'--model_architecture', default='low_latency_conv',
parser.add_argument('--model_architecture', type=str, default='conv',
                    help='What model architecture to use')
parser.add_argument('--check_nans', type=bool, default=False,
                    help='Whether to check for invalid numbers during processing')

FLAGS, unparsed = parser.parse_known_args()

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

# Start a new TensorFlow session.
sess = tf.InteractiveSession()

# Begin by making sure we have the training data we need. If you already have
# training data of your own, use `--data_url= ` on the command line to avoid
# downloading.
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
    FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
    FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
    FLAGS.data_url, FLAGS.data_dir, FLAGS.silence_percentage,
    FLAGS.unknown_percentage,
    FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
    FLAGS.testing_percentage, model_settings)
fingerprint_size = model_settings['fingerprint_size']
label_count = model_settings['label_count']
time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)

if os.path.exists(PSEUDO_LABEL_FILE):
    with open(PSEUDO_LABEL_FILE, 'rb') as fp:
        pseudo_label = pickle.load(fp)
    audio_processor.add_training_data(TEST_DIR, pseudo_label)
    used_pseudo_label = [sublist[0] for sublist in pseudo_label]

Words to work with :
{'sheila': True, 'seven': True, 'right': True, 'house': True, 'up': True, 'down': True, 'zero': True, 'go': True, 'yes': True, 'no': True, 'wow': True, 'six': True, 'two': True, 'bird': True, 'happy': True, 'marvin': True, 'stop': True, 'five': True, 'one': True, 'on': True, 'off': True, 'four': True, 'tree': True, 'dog': True, 'bed': True, 'cat': True, 'nine': True, 'three': True, 'eight': True, 'left': True}
Words in interest :
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
Training sample count : 22246
Testing sample count  : 3081
Validation sample count : 3093
Training sample count : 28246
Testing sample count  : 3081
Validation sample count : 3093


In [4]:
print (model_settings)
print (audio_processor)
print (fingerprint_size)
print (label_count)
print (time_shift_samples)
print (input_data.prepare_words_list(FLAGS.wanted_words.split(',')))

print (len(used_pseudo_label))
print (len(pseudo_label))

{'fingerprint_size': 3920, 'dct_coefficient_count': 40, 'sample_rate': 16000, 'window_size_samples': 480, 'spectrogram_length': 98, 'desired_samples': 16000, 'window_stride_samples': 160, 'label_count': 12}
3920
12
1600
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
6000
6000


In [5]:
# Figure out the learning rates for each training phase. Since it's often
# effective to have high learning rates at the start of training, followed by
# lower levels towards the end, the number of steps and learning rates can be
# specified as comma-separated lists to define the rate at each stage. For
# example --how_many_training_steps=10000,3000 --learning_rate=0.001,0.0001
# will run 13,000 training loops in total, with a rate of 0.001 for the first
# 10,000, and 0.0001 for the final 3,000.
training_steps_list = list(map(int, FLAGS.how_many_training_steps.split(',')))
learning_rates_list = list(map(float, FLAGS.learning_rate.split(',')))
if len(training_steps_list) != len(learning_rates_list):
    raise Exception(
        '--how_many_training_steps and --learning_rate must be equal length '
        'lists, but are %d and %d long instead' % (len(training_steps_list),
                                                   len(learning_rates_list)))

In [6]:
print (training_steps_list)
print (learning_rates_list)
print (FLAGS.model_architecture)

[1500, 1000, 1000, 1000]
[0.001, 0.0001, 1e-05, 1e-06]
conv


In [7]:
fingerprint_input = tf.placeholder(tf.float32, [None, fingerprint_size], name='fingerprint_input')
logits, dropout_prob = models.create_model(fingerprint_input, model_settings,
                                           FLAGS.model_architecture, is_training=True)
# Define loss and optimizer
ground_truth_input = tf.placeholder(tf.float32, [None, label_count], name='groundtruth_input')

# Optionally we can add runtime checks to spot when NaNs or other symptoms of
# numerical errors start occurring during training.
control_dependencies = []
if FLAGS.check_nans:
    checks = tf.add_check_numerics_ops()
    control_dependencies = [checks]

In [8]:
# Create the back propagation and training evaluation machinery in the graph.
with tf.name_scope('cross_entropy'):
    cross_entropy_mean = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_input, logits=logits))
tf.summary.scalar('cross_entropy', cross_entropy_mean)
    
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#control_dependencies.append(update_ops)
        
with tf.name_scope('train'), tf.control_dependencies(control_dependencies + update_ops):
    learning_rate_input = tf.placeholder(
        tf.float32, [], name='learning_rate_input')
    train_step = tf.train.AdamOptimizer(
        learning_rate_input).minimize(cross_entropy_mean)
    
predicted_indices = tf.argmax(logits, 1)
prediction_probabilities = tf.nn.softmax(logits)
expected_indices = tf.argmax(ground_truth_input, 1)
correct_prediction = tf.equal(predicted_indices, expected_indices)
confusion_matrix = tf.confusion_matrix(expected_indices, predicted_indices)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', evaluation_step)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [9]:
global_step = tf.train.get_or_create_global_step()
increment_global_step = tf.assign(global_step, global_step + 1)

saver = tf.train.Saver(tf.global_variables())

# Merge all the summaries and write them out to /tmp/retrain_logs (by default)
merged_summaries = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                   sess.graph)
validation_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/validation')

In [10]:
tf.global_variables_initializer().run()

start_step = 1

if FLAGS.start_checkpoint:
    models.load_variables_from_checkpoint(sess, FLAGS.start_checkpoint)
    start_step = global_step.eval(session=sess)

tf.logging.info('Training from step: %d ', start_step)

# Save graph.pbtxt.
tf.train.write_graph(sess.graph_def, FLAGS.train_dir,
                     FLAGS.model_architecture + '.pbtxt')

# Save list of words.
with gfile.GFile(
    os.path.join(FLAGS.train_dir, FLAGS.model_architecture + '_labels.txt'),
    'w') as f:
    f.write('\n'.join(audio_processor.words_list)+'\n')
    
print ("Word list : ")
print (audio_processor.words_list)

INFO:tensorflow:Training from step: 1 
Word list : 
['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']


In [11]:
def train_batch(training_step, learning_rate_value):
    # Pull the audio samples we'll use for training.
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        FLAGS.batch_size, 0, model_settings, FLAGS.background_frequency,
        FLAGS.background_volume, time_shift_samples, 'training', sess)
    #print ("{} Number of finger prints : {}".format(training_step, len(train_fingerprints)))
    # Run the graph with this batch of training data.
    train_summary, train_accuracy, cross_entropy_value, _, _ = sess.run(
        [
            merged_summaries, evaluation_step, cross_entropy_mean, train_step,
            increment_global_step
        ],
        feed_dict={
            fingerprint_input: train_fingerprints,
            ground_truth_input: train_ground_truth,
            learning_rate_input: learning_rate_value,
            dropout_prob: 0.5
        })
    train_writer.add_summary(train_summary, training_step)
    if (training_step % 10 == 0):
        tf.logging.info('Step #%d: rate %f, accuracy %.1f%%, cross entropy %f' %
                        (training_step, learning_rate_value, train_accuracy * 100,
                         cross_entropy_value))
    is_last_step = (training_step == training_steps_max)
    if (training_step % FLAGS.eval_step_interval) == 0 or is_last_step:
      set_size = audio_processor.set_size('validation')
      total_accuracy = 0
      total_conf_matrix = None
      for i in xrange(0, set_size, FLAGS.batch_size):
        validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(FLAGS.batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy, conf_matrix = sess.run(
            [merged_summaries, evaluation_step, confusion_matrix],
            feed_dict={
                fingerprint_input: validation_fingerprints,
                ground_truth_input: validation_ground_truth,
                dropout_prob: 1.0
            })
        validation_writer.add_summary(validation_summary, training_step)
        batch_size = min(FLAGS.batch_size, set_size - i)
        total_accuracy += (validation_accuracy * batch_size) / set_size
        if total_conf_matrix is None:
          total_conf_matrix = conf_matrix
        else:
          total_conf_matrix += conf_matrix
      tf.logging.info('Confusion Matrix:\n %s' % (total_conf_matrix))
      tf.logging.info('Step %d: Validation accuracy = %.1f%% (N=%d)' %
                      (training_step, total_accuracy * 100, set_size))

    # Save the model checkpoint periodically.
    if (training_step % FLAGS.save_step_interval == 0 or
        training_step == training_steps_max):
      checkpoint_path = os.path.join(FLAGS.train_dir,
                                     FLAGS.model_architecture + '.ckpt')
      tf.logging.info('Saving to "%s-%d"', checkpoint_path, training_step)
      saver.save(sess, checkpoint_path, global_step=training_step)
    
# Training loop.
training_steps_max = np.sum(training_steps_list)
for training_step in xrange(start_step, training_steps_max + 1):
    # Figure out what the current learning rate is.
    training_steps_sum = 0
    for i in range(len(training_steps_list)):
        training_steps_sum += training_steps_list[i]
        if training_step <= training_steps_sum:
            learning_rate_value = learning_rates_list[i]
            break
            
    train_batch(training_step, learning_rate_value)    



INFO:tensorflow:Step #10: rate 0.001000, accuracy 10.2%, cross entropy 5.544520
INFO:tensorflow:Step #20: rate 0.001000, accuracy 9.8%, cross entropy 3.533284
INFO:tensorflow:Step #30: rate 0.001000, accuracy 16.4%, cross entropy 2.520645
INFO:tensorflow:Step #40: rate 0.001000, accuracy 19.8%, cross entropy 2.241046
INFO:tensorflow:Step #50: rate 0.001000, accuracy 25.0%, cross entropy 2.141841
INFO:tensorflow:Step #60: rate 0.001000, accuracy 27.0%, cross entropy 2.014086
INFO:tensorflow:Step #70: rate 0.001000, accuracy 33.8%, cross entropy 1.908681
INFO:tensorflow:Step #80: rate 0.001000, accuracy 38.4%, cross entropy 1.895069
INFO:tensorflow:Step #90: rate 0.001000, accuracy 42.8%, cross entropy 1.668705
INFO:tensorflow:Step #100: rate 0.001000, accuracy 48.6%, cross entropy 1.588317
INFO:tensorflow:Saving to "/notebooks/data/training_pseudo_label/speech_commands_train/conv.ckpt-100"
INFO:tensorflow:Step #110: rate 0.001000, accuracy 48.0%, cross entropy 1.526533
INFO:tensorflow:S

INFO:tensorflow:Step 800: Validation accuracy = 86.0% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_pseudo_label/speech_commands_train/conv.ckpt-800"
INFO:tensorflow:Step #810: rate 0.001000, accuracy 87.6%, cross entropy 0.378694
INFO:tensorflow:Step #820: rate 0.001000, accuracy 90.6%, cross entropy 0.321206
INFO:tensorflow:Step #830: rate 0.001000, accuracy 88.0%, cross entropy 0.362382
INFO:tensorflow:Step #840: rate 0.001000, accuracy 88.2%, cross entropy 0.376211
INFO:tensorflow:Step #850: rate 0.001000, accuracy 89.2%, cross entropy 0.337452
INFO:tensorflow:Step #860: rate 0.001000, accuracy 86.0%, cross entropy 0.434882
INFO:tensorflow:Step #870: rate 0.001000, accuracy 87.6%, cross entropy 0.389703
INFO:tensorflow:Step #880: rate 0.001000, accuracy 89.8%, cross entropy 0.318991
INFO:tensorflow:Step #890: rate 0.001000, accuracy 89.0%, cross entropy 0.344501
INFO:tensorflow:Step #900: rate 0.001000, accuracy 89.0%, cross entropy 0.346998
INFO:tensorflow:Saving to

INFO:tensorflow:Step 1600: Validation accuracy = 90.7% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_pseudo_label/speech_commands_train/conv.ckpt-1600"
INFO:tensorflow:Step #1610: rate 0.000100, accuracy 93.2%, cross entropy 0.204966
INFO:tensorflow:Step #1620: rate 0.000100, accuracy 92.6%, cross entropy 0.218839
INFO:tensorflow:Step #1630: rate 0.000100, accuracy 92.4%, cross entropy 0.237861
INFO:tensorflow:Step #1640: rate 0.000100, accuracy 93.0%, cross entropy 0.236675
INFO:tensorflow:Step #1650: rate 0.000100, accuracy 93.6%, cross entropy 0.213439
INFO:tensorflow:Step #1660: rate 0.000100, accuracy 95.0%, cross entropy 0.189289
INFO:tensorflow:Step #1670: rate 0.000100, accuracy 93.6%, cross entropy 0.207288
INFO:tensorflow:Step #1680: rate 0.000100, accuracy 92.8%, cross entropy 0.231078
INFO:tensorflow:Step #1690: rate 0.000100, accuracy 92.6%, cross entropy 0.233090
INFO:tensorflow:Step #1700: rate 0.000100, accuracy 94.2%, cross entropy 0.200699
INFO:tensorfl

INFO:tensorflow:Step 2400: Validation accuracy = 91.1% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_pseudo_label/speech_commands_train/conv.ckpt-2400"
INFO:tensorflow:Step #2410: rate 0.000100, accuracy 94.8%, cross entropy 0.166687
INFO:tensorflow:Step #2420: rate 0.000100, accuracy 92.6%, cross entropy 0.198119
INFO:tensorflow:Step #2430: rate 0.000100, accuracy 94.4%, cross entropy 0.198215
INFO:tensorflow:Step #2440: rate 0.000100, accuracy 93.6%, cross entropy 0.261781
INFO:tensorflow:Step #2450: rate 0.000100, accuracy 95.2%, cross entropy 0.167952
INFO:tensorflow:Step #2460: rate 0.000100, accuracy 94.2%, cross entropy 0.200156
INFO:tensorflow:Step #2470: rate 0.000100, accuracy 96.2%, cross entropy 0.136178
INFO:tensorflow:Step #2480: rate 0.000100, accuracy 93.6%, cross entropy 0.194175
INFO:tensorflow:Step #2490: rate 0.000100, accuracy 92.8%, cross entropy 0.195345
INFO:tensorflow:Step #2500: rate 0.000100, accuracy 93.6%, cross entropy 0.189602
INFO:tensorfl

INFO:tensorflow:Step 3200: Validation accuracy = 91.4% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_pseudo_label/speech_commands_train/conv.ckpt-3200"
INFO:tensorflow:Step #3210: rate 0.000010, accuracy 93.8%, cross entropy 0.194169
INFO:tensorflow:Step #3220: rate 0.000010, accuracy 95.8%, cross entropy 0.142079
INFO:tensorflow:Step #3230: rate 0.000010, accuracy 93.2%, cross entropy 0.210690
INFO:tensorflow:Step #3240: rate 0.000010, accuracy 93.2%, cross entropy 0.190122
INFO:tensorflow:Step #3250: rate 0.000010, accuracy 96.0%, cross entropy 0.168109
INFO:tensorflow:Step #3260: rate 0.000010, accuracy 95.8%, cross entropy 0.156922
INFO:tensorflow:Step #3270: rate 0.000010, accuracy 94.2%, cross entropy 0.172604
INFO:tensorflow:Step #3280: rate 0.000010, accuracy 94.2%, cross entropy 0.171963
INFO:tensorflow:Step #3290: rate 0.000010, accuracy 94.2%, cross entropy 0.200612
INFO:tensorflow:Step #3300: rate 0.000010, accuracy 94.2%, cross entropy 0.202617
INFO:tensorfl

INFO:tensorflow:Step 4000: Validation accuracy = 91.2% (N=3093)
INFO:tensorflow:Saving to "/notebooks/data/training_pseudo_label/speech_commands_train/conv.ckpt-4000"
INFO:tensorflow:Step #4010: rate 0.000001, accuracy 93.8%, cross entropy 0.208372
INFO:tensorflow:Step #4020: rate 0.000001, accuracy 94.8%, cross entropy 0.177514
INFO:tensorflow:Step #4030: rate 0.000001, accuracy 96.4%, cross entropy 0.162512
INFO:tensorflow:Step #4040: rate 0.000001, accuracy 94.6%, cross entropy 0.171234
INFO:tensorflow:Step #4050: rate 0.000001, accuracy 92.0%, cross entropy 0.201552
INFO:tensorflow:Step #4060: rate 0.000001, accuracy 96.0%, cross entropy 0.154732
INFO:tensorflow:Step #4070: rate 0.000001, accuracy 94.6%, cross entropy 0.201598
INFO:tensorflow:Step #4080: rate 0.000001, accuracy 96.4%, cross entropy 0.147173
INFO:tensorflow:Step #4090: rate 0.000001, accuracy 94.2%, cross entropy 0.196479
INFO:tensorflow:Step #4100: rate 0.000001, accuracy 95.8%, cross entropy 0.175491
INFO:tensorfl

In [12]:
def print_test_info ():
    set_size = audio_processor.set_size('testing')
    tf.logging.info('set_size=%d', set_size)
    total_accuracy = 0
    total_conf_matrix = None
    for i in xrange(0, set_size, FLAGS.batch_size):
        test_fingerprints, test_ground_truth = audio_processor.get_data(
            FLAGS.batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
        test_accuracy, conf_matrix = sess.run(
            [evaluation_step, confusion_matrix],
            feed_dict={
                fingerprint_input: test_fingerprints,
                ground_truth_input: test_ground_truth,
                dropout_prob: 1.0
            })
        batch_size = min(FLAGS.batch_size, set_size - i)
        total_accuracy += (test_accuracy * batch_size) / set_size
        if total_conf_matrix is None:
            total_conf_matrix = conf_matrix
        else:
            total_conf_matrix += conf_matrix
    tf.logging.info('Confusion Matrix:\n %s' % (total_conf_matrix))
    tf.logging.info('Final test accuracy = %.1f%% (N=%d)' % (total_accuracy * 100,
                                                         set_size))
    
print_test_info ()

INFO:tensorflow:set_size=3081
INFO:tensorflow:Confusion Matrix:
 [[257   0   0   0   0   0   0   0   0   0   0   0]
 [  0 199   0   0   8  18   3  12   4   1   5   7]
 [  0   4 235   0   0   6   6   3   0   0   2   0]
 [  1   2   0 216   1  19   2   4   0   0   2   5]
 [  0   6   0   0 258   1   0   1   1   1   3   1]
 [  1   2   0   6   0 239   2   0   0   1   0   2]
 [  0   1   3   0   3   2 257   1   0   0   0   0]
 [  1  11   0   1   1   0   1 243   0   0   1   0]
 [  0   4   0   0   1   2   1   2 234   2   0   0]
 [  0   3   0   0  17   1   2   2   8 227   2   0]
 [  0   0   0   0   5   3   2   2   0   0 237   0]
 [  0  11   0  19   4   9   6   2   0   1   2 197]]
INFO:tensorflow:Final test accuracy = 90.8% (N=3081)


### Use the trained model to make a prediction

In [ ]:
training_step = start_step
learning_rate_value = learning_rates_list[-1]
print start_step
print training_step


In [13]:
used_pseudo_label = []

def get_new_pseudo_label (subm, used_pseudo_label, num_label_per_category = 400):
    data = pd.DataFrame(subm)
    data.columns = ['filename', 'label', 'prob']
    data = data.sort_values(['label', 'prob'], ascending=False)

    new_pseudo_label = []
    pseudo_label = []
    # pick highest 200 entries
    for label in audio_processor.words_list[2:]:
        new_data = data[data['label'] == label]
        new_data = new_data[~new_data['filename'].isin(used_pseudo_label)].drop(columns=['prob'])
        pseudo_label += new_data.head(num_label_per_category).values.tolist()
        new_pseudo_label += new_data.drop(columns=['label']).head(num_label_per_category).values.tolist()
    new_pseudo_label = [val for sublist in new_pseudo_label for val in sublist]    
    return pseudo_label, new_pseudo_label

def get_prediction(testfile_dir):
    os.chdir (testfile_dir)
    print (os.getcwd())

    file_names = os.listdir(testfile_dir)
    set_size = len(file_names)
    subm = zip(file_names, ('silence' for x in range(set_size)))
    
    # process the audio files
    batch_num = 0
    for i in xrange(0, set_size, FLAGS.batch_size):
        batch_size = min(FLAGS.batch_size, set_size - i)
        batch_filenames = file_names[i:i+batch_size]
        predict_fingerprints = audio_processor.get_predicion_data(
            batch_filenames, model_settings, sess)

        predict_idx, predict_prob = sess.run(
            [predicted_indices, prediction_probabilities],
            feed_dict={
                fingerprint_input: predict_fingerprints,
                #ground_truth_input: test_ground_truth,
                dropout_prob: 1.0
            })

        '''
        if batch_num % 100 == 0:
            print ("i {}, batch num {}, batch size = {}".format(i, batch_num, batch_size))
            print (batch_filenames[10:20])
            print (predict_idx[10:20])
            print (predict_prob[10:20])
            print (predict_prob[:][predict_idx])
        '''

        for j in range(batch_size):
            subm[i+j] = (subm[i+j][0], audio_processor.words_list[predict_idx[j]], predict_prob[j][predict_idx[j]])

        batch_num += 1
    
    return subm

In [ ]:
#new_data.drop(columns=['prob']).values.tolist()
print (pseudo_label[199])
print (pseudo_label[399])
print (pseudo_label[599])
print (pseudo_label[799])

print (len(pseudo_label))
print (pseudo_label[:10])
#print (len(previous_pseudo_label))
#print (previous_pseudo_label[:10])
print (learning_rate_value)
#previous_pseudo_label += pseudo_label

In [14]:
subm = get_prediction(TEST_DIR)

print ("Number of pseudo labeled data : {}".format(len(used_pseudo_label)))

/notebooks/data/test/audio
Number of pseudo labeled data : 0


In [ ]:
# train more after adding pseudo labeled data
pseudo_label, used_label = get_new_pseudo_label (subm, used_pseudo_label)

used_pseudo_label += used_label

audio_processor.add_training_data(TEST_DIR, pseudo_label)

start_step = global_step.eval(session=sess)
for training_step in xrange(start_step, start_step + 1000):
    # Use last used learning rate
    train_batch(training_step, learning_rate_value)
    
print_test_info ()

In [ ]:
#new_data.drop(columns=['prob']).values.tolist()
print (pseudo_label[199])
print (pseudo_label[399])
print (pseudo_label[599])
print (pseudo_label[799])

print (len(pseudo_label))
print (learning_rate_value)

In [ ]:
print ("Number of pseudo labeled data : {}".format(len(used_pseudo_label)))

# train more after adding pseudo labeled data
start_step = training_step
for training_step in xrange(start_step, start_step + 1000):
    # Use last used learning rate            
    train_batch(training_step, learning_rate_value*10)
print_test_info ()

In [ ]:
print(pseudo_label[:10])
print([sublist[0] for sublist in pseudo_label])

In [ ]:
with open(DATA_DIR + '/test/pseudo_label.pickle', 'wb') as fp:
    pickle.dump(pseudo_label, fp)

### Create submission file

In [15]:
import csv

# submission1.csv : sample submission
# submission2.csv : TensorFlow example code prediction, trained with Speech Commands Data Set v0.01
# submission3.csv : TensorFlow example code prediction, trained with Kaggle Data Set
# submission4.csv : TensorFlow example code prediction, trained using AtomOptimizer
# submission5.csv : TensorFlow example code prediction, using Xavier initialization
# submission6.csv : TensorFlow example code prediction, using batch normalization
# submission7.csv : TensorFlow example code prediction, using pseudo label
# submission8.csv : TensorFlow example code prediction, using low latency model

submission_file_name = DATA_DIR+'/submission7.csv'
print (submission_file_name)
#np.savetxt(submission_file_name, subm, header='fname,label', comments='')
with open(submission_file_name, 'wb') as outfile:
    wr = csv.writer(outfile)
    wr.writerow(('fname', 'label'))
    for i in range(len(subm)):
        wr.writerow(subm[i][:2])

/notebooks/data/submission7.csv


In [16]:
from IPython.display import FileLink
FileLink(submission_file_name)

/notebooks/data/submission7.csv